# ___Imports___

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
import plotly.express as px
from geopy.geocoders import Nominatim
import geopy as gp
from datetime import datetime
from glob import glob

## ___NASA data___

In [14]:
nasa_data = pd.read_csv('../data/meteorite-landings.csv')
print(nasa_data.columns)

nasa_data.head()
np.shape(nasa_data)

Index(['name', 'id', 'nametype', 'recclass', 'mass', 'fall', 'year', 'reclat',
       'reclong', 'GeoLocation'],
      dtype='object')


(45716, 10)

## ___IMO data___

In [12]:
imo_data = pd.read_csv('../data/IMO/data/IMO9920.csv')
print(imo_data.columns)

imo_data.head()
np.shape(imo_data)

Index(['Session ID', 'Start Date', 'Observer ID', 'Submitter ID',
       'Actual Observer Name', 'Submitted by', 'City', 'Country', 'Latitude',
       'Longitude', 'Elevation'],
      dtype='object')


(42989, 11)